In [1]:
%load_ext sql
%sql sqlite:///iris.db

# Activity 6-2
------
In this activity, we will practice bulk loading a data file and creating views. The file we will bulk load contains the [`Iris` plant measurements](https://archive.ics.uci.edu/ml/datasets/Iris) data from the UCI Machine Learning repository. The data is already in our repo, in the file `iris.csv`, in comma-separated values format.

The data have the following attributes:
1. sepal length in cm (real)
2. sepal width in cm (real)
3. petal length in cm (real)
4. petal width in cm (real)
5. class (string)

## Exercise 1

Create the `Iris` table that will contain the attributes above, then use bulk-loading to lead data into the table.

In [2]:
%%sql
DROP TABLE IF EXISTS Iris;
CREATE TABLE Iris (
    sepal_length REAL,
    sepal_width REAL,
    petal_length REAL,
    petal_width REAL,
    class VARCHAR,
    UNIQUE(sepal_length, sepal_width, petal_length, petal_width, class)
);

.separator ","
.import iris.csv Iris

Done.
Done.
(sqlite3.OperationalError) near ".": syntax error [SQL: u'.separator ","\n.import iris.csv Iris']


In [3]:
%sql select count(*) from Iris

Done.


count(*)
0


As you no doubt had found out, bulk-loading does not work as-is in Jupyter Notebook, as it is not part of the DDL or DQL in SQLite. Instead, they are command-line utilities which can be accessed via the `sqlite3` command line.

An alternative to using those commands is to load the data from the file via Python and insert the rows into the table. Here's a shortcut:

In [4]:
import sqlite3 as lite
import csv
import os

my_file = open('iris.csv', 'r')
reader = csv.reader(my_file, delimiter=',')

con = lite.connect('iris.db', isolation_level = 'exclusive')
cur  = con.cursor()

cur.execute("DROP TABLE IF EXISTS Iris")
cur.execute(r"""
CREATE TABLE IF NOT EXISTS Iris (
    sepal_length REAL,
    sepal_width REAL,
    petal_length REAL,
    petal_width REAL,
    class VARCHAR
);
""")

for row in reader:
    cur.execute("INSERT INTO Iris VALUES(?, ?, ?, ?, ?)", row)

con.commit()


In [5]:
%sql select count(*) from Iris

Done.


count(*)
150


In [6]:
%sql SELECT * FROM Iris LIMIT 5;

Done.


sepal_length,sepal_width,petal_length,petal_width,class
5.1,3.5,1.4,0.2,Iris-setosa
4.9,3.0,1.4,0.2,Iris-setosa
4.7,3.2,1.3,0.2,Iris-setosa
4.6,3.1,1.5,0.2,Iris-setosa
5.0,3.6,1.4,0.2,Iris-setosa


## Exercise 2

Create a view of the data where the class does not contain the flower name "Iris-", named `IrisView`. You can use the [`SUBSTR`](https://www.sqlite.org/lang_corefunc.html#substr) function to keep only a subset of the `class` attribute values. Can you capitalize the class name?

In [7]:
%%sql
DROP VIEW IF EXISTS IrisView;
CREATE VIEW IrisView AS
SELECT sepal_length, sepal_width, petal_length, petal_width, UPPER(SUBSTR(class, 6, 1)) || SUBSTR(class,7) AS class
FROM Iris;

Done.
Done.


[]

In [8]:
%sql SELECT DISTINCT class from IrisView;

Done.


class
Setosa
Versicolor
Virginica


## Exercise 3

Create a view, named `IrisSetosa`, containing only unique sepal and petal measurements for Iris Setosa flowers. You can either retrieve records from the Iris table or the IrisView view.

In [9]:
%%sql 
DROP VIEW IF EXISTS IrisSetosa;
CREATE VIEW IrisSetosa AS
SELECT sepal_length, sepal_width, petal_length, petal_width
FROM IrisView
WHERE class = 'Setosa';

Done.
Done.


[]

In [10]:
%sql SELECT count(*) FROM IrisSetosa;

Done.


count(*)
50
